In [1]:
from cluster.RayJobManager import RayJobManager

In [2]:
RAY_ADDRESS  = "http://10.254.31.202:8265"
#RAY_ADDRESS  = "local"
ENTRYPOINT   = "python3 cluster/tune_grid.py"
WORKING_DIR  = "/eos/user/e/ecoradin/GitHub/TrackSelectorDNN-Ray/"
PIP = {
    "packages": [
        "torch",
        "ray[tune]",
        "git+https://github.com/EmanueleCoradin/TrackSelectorDNN.git@main"
    ],
    "pip_check": False,
    "pip_install_options": ["--vvv"]
}

ray_manager = RayJobManager(ray_address=RAY_ADDRESS)

In [3]:
job_id = ray_manager.submit_job(
    entrypoint=ENTRYPOINT,
    runtime_env={
        "working_dir": WORKING_DIR, 
        "pip": PIP, 
    }
    
)

ray_manager.wait_until_done(job_id)

2025-11-04 15:27:58,492	INFO dashboard_sdk.py:338 -- Uploading package gcs://_ray_pkg_9902c00d17eb191f.zip.
2025-11-04 15:27:58,495	INFO packaging.py:588 -- Creating a file package for local module '/eos/user/e/ecoradin/GitHub/TrackSelectorDNN-Ray/'.


Submitted Ray job: raysubmit_h4wNperMfe39VG8A
Job status: PENDING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: SUCCEEDED

--- Job Logs ---
2025-11-04 07:27:59,329	INFO job_manager.py:568 -- Runtime env is setting up.
2025-11-04 07:28:01,875	INFO worker.py:1692 -- Using address 10.100.192.237:6379 set in the environment variable RAY_ADDRESS
2025-11-04 07:28:01,877	INFO worker.py:1833 -- Connecting to existing Ray cluster at address: 10.100.192.237:6379...
2025-11-04 07:28:01,887	INFO worker.py:2004 -- Connected to Ray cluster. View the dashboard at 10.100.192.237:8265 
/home/ray/anaconda3/lib/python3.9/site-packages/ray/_private/worker.py:2052: FutureWarning: Tip: In future versions of Ray, Ray will no longer override accelerator visible devices env var if num_gpus=0 or num_gpus=None (default). To enable this behavior and turn off this error message, set RAY_ACCEL_ENV_VAR_OVERRIDE_ON_ZERO=0
  warnings

In [1]:
!pip uninstall -y TrackSelectorDNN

Found existing installation: TrackSelectorDNN 0.1
Uninstalling TrackSelectorDNN-0.1:
  Successfully uninstalled TrackSelectorDNN-0.1


In [2]:
!pip install -e ../TrackSelectorDNN

Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///eos/home-i03/e/ecoradin/GitHub/TrackSelectorDNN
  Preparing metadata (setup.py) ... done
  DEPRECATION: Legacy editable install of TrackSelectorDNN==0.1 from file:///eos/home-i03/e/ecoradin/GitHub/TrackSelectorDNN (setup.py develop) is deprecated. pip 25.0 will enforce this behaviour change. A possible replacement is to add a pyproject.toml or enable --use-pep517, and use setuptools >= 64. If the resulting installation is not behaving as expected, try using --config-settings editable_mode=compat. Please consult the setuptools documentation for more information. Discussion can be found at https://github.com/pypa/pip/issues/11457
  Running setup.py develop for TrackSelectorDNN


In [3]:
'''
Uncomment to run locally
from ray import tune
from TrackSelectorDNN.configs.schema import load_config
from TrackSelectorDNN.tune.trainable import trainable

base_cfg = load_config("base.yaml").model_dump()

search_space = {
    **base_cfg,  # include all validated defaults
    "lr": tune.loguniform(1e-5, 1e-2),
    "weight_decay": tune.loguniform(1e-5, 1e-2),
    "latent_dim": tune.choice([16, 32, 64]),
    "pooling_type": tune.choice(["sum", "mean", "softmax"]),
    "netA_hidden_layers": tune.choice([1, 2, 3, 4, 5]),
    "netB_hidden_layers": tune.choice([1, 2, 3, 4, 5]),
}


tuner = tune.Tuner(
    trainable,
    param_space=search_space,
    tune_config=tune.TuneConfig(metric="val_loss", mode="min", num_samples=1),
)

results = tuner.fit()'''

(trainable pid=11198) [Epoch 1/10] train_loss=0.1704, val_loss=0.0606, val_acc=0.9944


(trainable pid=11198) /eos/user/e/ecoradin/.local/lib/python3.11/site-packages/ray/train/_internal/session.py:792: RayDeprecationWarning: `ray.train.report` should be switched to `ray.tune.report` when running in a function passed to Ray Tune. This will be an error in the future. See this issue for more context: https://github.com/ray-project/ray/issues/49454
(trainable pid=11198)   _log_deprecation_warning(
(trainable pid=11198) /eos/user/e/ecoradin/.local/lib/python3.11/site-packages/ray/tune/trainable/trainable_fn_utils.py:41: RayDeprecationWarning: The `Checkpoint` class should be imported from `ray.tune` when passing it to `ray.tune.report` in a Tune function. Please update your imports. See this issue for more context and migration options: https://github.com/ray-project/ray/issues/49454. Disable these warnings by setting the environment variable: RAY_TRAIN_ENABLE_V2_MIGRATION_WARNINGS=0
(trainable pid=11198)   _log_deprecation_warning(
(trainable pid=11198) Checkpoint successful

(trainable pid=11198) [Epoch 2/10] train_loss=0.0471, val_loss=0.0291, val_acc=0.9959


(trainable pid=11198) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/eos/user/e/ecoradin/ray_results/trainable_2025-11-04_15-47-52/trainable_ac5fb_00000_0_latent_dim=64,lr=0.0001,netA_hidden_layers=3,netB_hidden_layers=1,pooling_type=sum,weight_decay=0.0000_2025-11-04_15-48-14/checkpoint_000001)


(trainable pid=11198) [Epoch 3/10] train_loss=0.0274, val_loss=0.0194, val_acc=0.9959


...

(trainable pid=11198) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/eos/user/e/ecoradin/ray_results/trainable_2025-11-04_15-47-52/trainable_ac5fb_00000_0_latent_dim=64,lr=0.0001,netA_hidden_layers=3,netB_hidden_layers=1,pooling_type=sum,weight_decay=0.0000_2025-11-04_15-48-14/checkpoint_000007)


(trainable pid=11198) [Epoch 9/10] train_loss=0.0142, val_loss=0.0109, val_acc=0.9966


(trainable pid=11198) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/eos/user/e/ecoradin/ray_results/trainable_2025-11-04_15-47-52/trainable_ac5fb_00000_0_latent_dim=64,lr=0.0001,netA_hidden_layers=3,netB_hidden_layers=1,pooling_type=sum,weight_decay=0.0000_2025-11-04_15-48-14/checkpoint_000008)


(trainable pid=11198) [Epoch 10/10] train_loss=0.0133, val_loss=0.0105, val_acc=0.9967
(trainable pid=11198) [DEBUG] Final report to Ray: /eos/user/e/ecoradin/GitHub/TrackSelectorDNN/runs/2025-11-04_15-48-29


(trainable pid=11198) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/eos/user/e/ecoradin/ray_results/trainable_2025-11-04_15-47-52/trainable_ac5fb_00000_0_latent_dim=64,lr=0.0001,netA_hidden_layers=3,netB_hidden_layers=1,pooling_type=sum,weight_decay=0.0000_2025-11-04_15-48-14/checkpoint_000009)
(trainable pid=11198) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/eos/user/e/ecoradin/ray_results/trainable_2025-11-04_15-47-52/trainable_ac5fb_00000_0_latent_dim=64,lr=0.0001,netA_hidden_layers=3,netB_hidden_layers=1,pooling_type=sum,weight_decay=0.0000_2025-11-04_15-48-14/checkpoint_000010)
2025-11-04 16:13:18,311	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/eos/user/e/ecoradin/ray_results/trainable_2025-11-04_15-47-52' in 0.2468s.
2025-11-04 16:13:18,326	INFO tune.py:1041 -- Total run time: 1504.18 seconds (1503.38 seconds for the tuning loop).


In [4]:
best_result = results.get_best_result(metric="val_loss", mode="min")

print("Best config found:")
print(best_result.config)

best_ckpt_dir = best_result.checkpoint
print("Best checkpoint path:", best_ckpt_dir)

Best config found:
{'hit_input_dim': 10, 'track_feat_dim': 16, 'latent_dim': 64, 'pooling_type': 'sum', 'netA_hidden_dim': 32, 'netA_hidden_layers': 3, 'netA_batchnorm': True, 'netA_activation': 'silu', 'netB_hidden_dim': 64, 'netB_hidden_layers': 1, 'netB_batchnorm': True, 'netB_activation': 'silu', 'lr': 8.411694872074715e-05, 'epochs': 10, 'batch_size': 64, 'dataset_type': 'dummy', 'dummy_load_path': '/eos/user/e/ecoradin/GitHub/TrackSelectorDNN/TrackSelectorDNN/data/dummy_dataset.pt', 'train_path': 'data/processed/tracks_train.pt', 'n_tracks': 100000, 'max_hits': 15, 'val_fraction': 0.2, 'weight_decay': 2.0040837755536215e-05}
Best checkpoint path: Checkpoint(filesystem=local, path=/eos/user/e/ecoradin/ray_results/trainable_2025-11-04_15-47-52/trainable_ac5fb_00000_0_latent_dim=64,lr=0.0001,netA_hidden_layers=3,netB_hidden_layers=1,pooling_type=sum,weight_decay=0.0000_2025-11-04_15-48-14/checkpoint_000010)


In [6]:
import json
import os
import torch
from TrackSelectorDNN.models.track_classifier import TrackClassifier

run_root = "/eos/user/e/ecoradin/GitHub/TrackSelectorDNN/runs"
best_val = float("inf")
best_ckpt_path = None
best_config = None

for trial_name in os.listdir(run_root):
    trial_dir = os.path.join(run_root, trial_name)
    metrics_file = os.path.join(trial_dir, "best_metrics.json")
    ckpt_file = os.path.join(trial_dir, "best_model.pt")

    if os.path.exists(metrics_file):
        with open(metrics_file, "r") as f:
            metrics = json.load(f)
        if metrics["val_loss"] < best_val:
            best_val = metrics["val_loss"]
            best_ckpt_path = ckpt_file
            # optionally save config
            config_file = os.path.join(trial_dir, "config.yaml")
            best_config = config_file if os.path.exists(config_file) else None

print("Best checkpoint:", best_ckpt_path)
print("Best val_loss:", best_val)
print("Best config file:", best_config)

Best checkpoint: /eos/user/e/ecoradin/GitHub/TrackSelectorDNN/runs/2025-11-04_15-48-29/best_model.pt
Best val_loss: 0.010471555917942896
Best config file: /eos/user/e/ecoradin/GitHub/TrackSelectorDNN/runs/2025-11-04_15-48-29/config.yaml


TypeError: TrackSelectorDNN.models.track_classifier.TrackClassifier() argument after ** must be a mapping, not str